In [1]:
from PIL import Image
import numpy as np
from numpy import asarray
from numpy import expand_dims
from keras.models import load_model
import pickle
import cv2
from datetime import datetime
from mtcnn.mtcnn import MTCNN 

In [2]:
FaceNetModel = load_model('facenet_keras.h5') #loading faceNet model
faceDetector = MTCNN() 

2022-07-11 13:50:27.005470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [4]:
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

In [5]:
vid_input =cv2.VideoCapture(0) #0= one camera
while(1):
    _,image= vid_input.read() 
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_array=asarray(image)
    mtcnn_results = faceDetector.detect_faces(image)
    
    if mtcnn_results == []:
        continue 
        
    x1, y1, width, height = mtcnn_results[0]['box']
    
    x1=abs(x1)
    y1=abs(y1)
    x2=x1+width
    y2=y1+height
    
    face = image[y1:y2, x1:x2] 
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)

    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = FaceNetModel.predict(face)
    
    min_dist=10
    identity=' '
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            markAttendance(identity)
            #camera input sahi karna hai
            
    cv2.putText(image,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(image,(x1,y1),(x2,y2), (0,255,0), 2)
        
    cv2.imshow('res',image)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 20ms/step


14/14 [==============================] - 0s 5ms/step


KeyboardInterrupt: 